In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.data import astronaut
from skimage.color import label2rgb, rgb2gray
from skimage.filters import sobel
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import measure, data, io, segmentation, color, img_as_ubyte
from skimage.future import graph
#from google.colab import drive

In [ ]:
H = cv2.imread('/content/drive/MyDrive/image segmentation/-x.png')
print(H.shape)
plt.imshow(H)
plt.axis('off')
plt.show()

In [ ]:
# convert image to grayscale
gray_H = cv2.cvtColor(H, cv2.COLOR_BGR2GRAY)

# Histogram equalization
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
gray_cl1 = clahe.apply(gray_H)

plt.imshow(gray_cl1)
plt.show()

# create RGB image with grayscale intensity values
rgb_img_0 = np.zeros((gray_cl1.shape[0], gray_cl1.shape[1], 3), dtype=np.uint8)
rgb_img_0[:, :, 0] = gray_cl1
rgb_img_0[:, :, 1] = gray_cl1
rgb_img_0[:, :, 2] = gray_cl1

# Blur image
#median_H = cv2.medianBlur(rgb_img_0, 1)
# micron pin = 3, Nick = 5

# Display the grayscale image
plt.imshow(median_H)
plt.show()

In [ ]:
# Quickshift superpixel segmentation (over-segmented)
labels = quickshift(median_H, kernel_size=3, max_dist=200, ratio=0.9)

out1 = color.label2rgb(labels, median_H, kind='avg', bg_label=0)
out1 = segmentation.mark_boundaries(out1, labels, (255, 255, 0))
plt.axis('off')
plt.imshow(out1)
plt.show()

In [ ]:
## RAG Merging to merge over-segmented image
def _weight_mean_color(graph, src, dst, n):
    diff = graph.nodes[dst]['mean color'] - graph.nodes[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}

def merge_mean_color(graph, src, dst):
    graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
    graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
    graph.nodes[dst]['mean color'] = (graph.nodes[dst]['total color'] /
                                      graph.nodes[dst]['pixel count'])

g = graph.rag_mean_color(median_H, labels)

labels2 = graph.merge_hierarchical(labels, g, thresh=9, rag_copy=False,
                                   in_place_merge=True,
                                   merge_func=merge_mean_color,
                                   weight_func=_weight_mean_color)
# Variable adjustment needed: bigger thresh for larger grain
# opensource =11

# output the label mask
plt.imshow(labels2)
out2 = color.label2rgb(labels2, median_H, kind='avg', bg_label=0)
col = color.label2rgb(labels2, bg_label=0)
plt.axis('off')
plt.imshow(col)
plt.show()
# mark the boundary of the mask
out_b = segmentation.mark_boundaries(out2, labels2, (255, 255, 0))
#out2 = segmentation.find_boundaries(out2)
plt.axis('off')
plt.imshow(out_b)
plt.show()
plt.axis('off')
plt.imshow(out2)
plt.show()

In [ ]:
from skimage import measure, io, img_as_ubyte
import matplotlib.pyplot as plt
from skimage.color import label2rgb, rgb2gray
import numpy as np

props_H = measure.regionprops_table(labels2, H,
                          properties=['label',
                                      'area', 'equivalent_diameter',
                                      'mean_intensity', 'solidity',
                                      'MajorAxisLength','MinorAxisLength',
                                      'centroid', 'orientation',
                                      'feret_diameter_max', 'perimeter',
                                      'perimeter_crofton', 'intensity_max',
                                      'intensity_mean', 'intensity_min'])

import pandas as pd
df_H = pd.DataFrame(props_H)
print(df_H)